In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
articles=pd.read_csv('h-and-m-personalized-fashion-recommendations/articles.csv', dtype={"article_id": "str"})
customers=pd.read_csv('h-and-m-personalized-fashion-recommendations/customers.csv')
transactions=pd.read_csv('h-and-m-personalized-fashion-recommendations/transactions_train.csv', dtype={"article_id": "str"},index_col='t_dat')
transactions.head()

In [ ]:
customer = transactions.customer_id.unique()
customer_map = {i:val for i,val in enumerate(customer)}
inverse_customer_map = {val:i for i,val in enumerate(customer)}
article = transactions.article_id.unique()
article_map = {i:val for i,val in enumerate(article)}
inverse_article_map = {val:i for i,val in enumerate(article)}
transactions['customer_id']=transactions['customer_id'].map(inverse_customer_map)
transactions['old_article_id']=transactions['article_id']
transactions['article_id']=transactions['article_id'].map(inverse_article_map)

In [ ]:
train=transactions['2018-01-01':'2019-01-01']
test=transactions['2018-01-01':'2019-01-01']
train

In [ ]:
user_id_input = keras.layers.Input(shape=[1], name='user')
item_id_input = keras.layers.Input(shape=[1], name='item')
embedding_size = 10 # 5
user_embedding = keras.layers.Embedding(output_dim=embedding_size, input_dim=customer.shape[0],
                           input_length=1, name='user_embedding')(user_id_input)
item_embedding = keras.layers.Embedding(output_dim=embedding_size, input_dim=article.shape[0],
                           input_length=1, name='item_embedding')(item_id_input)
user_vecs = keras.layers.Reshape([embedding_size])(user_embedding)
item_vecs = keras.layers.Reshape([embedding_size])(item_embedding)
input_vecs = keras.layers.Concatenate()([user_vecs, item_vecs])
x = keras.layers.Dense(128, activation='relu')(input_vecs)
y = keras.layers.Dense(1)(x)
model = keras.models.Model(inputs=[user_id_input, item_id_input], outputs=y)
model.compile(optimizer='adam', loss='mse')
history = model.fit([train["customer_id"], train["article_id"]]
                    , train["price"]
                    , batch_size=64, epochs=10, validation_split=0.1)